# Singularityの利用

---

構築したOpenHPC環境で[Singularity](https://sylabs.io/singularity/)を利用します。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* Singularityがロードされる設定になっている
* OpenHPC環境のマスターノードに対してSSHでログインできる

> この Notebookでは、管理ユーザ以外が実行した状況に合わせるために Ansible ではなく ssh で操作を行います。

マスターノードに対して SSH でログインできることを確認します。マスターノードのアドレスを指定してください。

In [ ]:
# (例)
# master_address = '172.30.XXX.xxx'

master_address =

SSHでログインするユーザ名を指定してください。

In [ ]:
# (例)
# user = 'vcp'

user = 'vcp'

必要であればSSHの秘密鍵を指定してください。

In [ ]:
# (例)
# ssh_identity = '~/.ssh/id_rsa'

SSHでログインする際の引数を、変数に格納しておきます。

In [ ]:
# ユーザ名とホスト名
target = f'{user}@{master_address}'
print(target)

# SSHのコマンドライン引数
ssh_opts = f'-i {ssh_identity}' if 'ssh_identity' in vars() else ''
print(ssh_opts)

マスターノードに対してSSHでログインしてコマンドを実行してみます。

In [ ]:
!ssh {ssh_opts} {target} hostname

Singularityがロードされていることを確認します。

In [ ]:
!ssh {ssh_opts} {target} module is-loaded singularity

## Singularityの実行

singularityで docker コンテナイメージを実行してみます。

> singularityの操作の詳細については Singularityの[User Guide](https://sylabs.io/guides/3.4/user-guide/index.html)などを参照してください。

dockerhub にある CentOS のコンテナイメージを singularity で実行してみます。まずコンテナイメージを取得してみます。

> 数分程度かかります。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c "singularity\ pull\ docker://centos:8"

コンテナを実行して `/etc/redhat-release` の内容を表示してみます。

In [ ]:
!ssh {ssh_opts} {target} bash -l -c "singularity\ exec\ docker://centos:8\ cat\ /etc/redhat-release"

SLURMで実行してみます。

まず、実行するノード数を指定してください。

In [ ]:
# (例)
# num_nodes = 2

num_nodes =

srunで実行します。

In [ ]:
cmd = 'singularity\\ exec\\ docker://centos:8\\ cat\\ /etc/redhat-release'
!ssh {ssh_opts} {target} bash -l -c "env\ XDG_RUNTIME_DIR=\ LC_ALL=\ srun\ -l\ -N\ {num_nodes}\ {cmd}"